## USPTO ODP C# Notebook — Retrieving IP Data

Reference: [Getting Started — USPTO Open Data APIs](https://data.uspto.gov/apis/getting-started)


In [ ]:
// load nuget packages
#r "nuget: DotNetEnv, 2.5.*"
#r "nuget: System.Net.Http.Json, 6.0.0"

// load nuget packages for Azure Key Vault
#r "nuget: Azure.Identity, 1.5.0"
#r "nuget: Azure.Security.KeyVault.Secrets, 4.3.0"

// import namespaces
using System.Collections.Generic;
using System.Linq;
using System.Net.Http;
using System.Net.Http.Headers;
using DotNetEnv;
using System.Text.Json;
using System.Threading.Tasks;
using Azure.Identity;
using Azure.Security.KeyVault.Secrets;

In [ ]:
// get api key from Azure Key Vault. the secret name is "USPTO-ApiKey"
var kvVaultUri = "https://tdg-kv.vault.azure.net/";
var secretName = "USPTO-ApiKey";
var credential = new DefaultAzureCredential();

var kvClient = new SecretClient(new Uri(kvVaultUri), credential);
KeyVaultSecret secret = kvClient.GetSecret(secretName);
string apiKey = secret.Value;
Console.WriteLine($"API Key from Azure Key Vault: {apiKey}");

In [ ]:
// define base URL for USPTO API
const string BaseUrl = "https://api.uspto.gov/api/v1/patent/applications/";

// load environment variables from .env file
Env.Load();
var apiKey = Environment.GetEnvironmentVariable("ODP_API_KEY");
Console.WriteLine($"Using API Key: {apiKey}");

// configure HttpClient
var client = new HttpClient();
client.BaseAddress = new Uri(BaseUrl);
client.DefaultRequestHeaders.Accept.Clear();
client.DefaultRequestHeaders.Accept.Add(new MediaTypeWithQualityHeaderValue("application/json"));
client.DefaultRequestHeaders.Add("X-API-KEY", apiKey);

In [ ]:
Console.WriteLine($"Using API Key: {apiKey}");

In [ ]:
// retrieve patent application number information
var applicationNumber = "16892097";
var response = await client.GetAsync($"{applicationNumber}");
if (response.IsSuccessStatusCode)
{
    var data = await response.Content.ReadAsStringAsync();
    Console.WriteLine(data);
    // retrieve application Name from data (parsing JSON is omitted for brevity)
    var appName = JsonDocument.Parse(data).RootElement.Display(); //.GetProperty("applicationNameText").GetString();
    Console.WriteLine($"Application Name: {appName}");
    // var appNum = JsonDocument.Parse(data).RootElement.GetProperty("applicationNameText").GetString();
    // Console.WriteLine($"Application Number: {appNum}");
}
else
{
    Console.WriteLine($"Error: {response.StatusCode}");
}